In [1]:
import numpy as np
import pandas as pd
import pylab as p
import pymap3d
from numpy.linalg import inv
import warnings
from gnatss.loaders import read_novatel_L1_data_files
from gnatss.loaders import load_gps_solutions

warnings.filterwarnings('ignore')

In [2]:
inspvaa_df = read_novatel_L1_data_files(data_files=['tests/data/2022/NCL1/NCL1_INSPVAA.dat'], data_format="INSPVAA")

In [3]:
insstdeva_df = read_novatel_L1_data_files(data_files=['tests/data/2022/NCL1/NCL1_INSSTDEVA.dat'], data_format="INSSTDEVA")
 

In [4]:
inspvaa_df = inspvaa_df.rename(
    columns={
        "ant_e": "vel_e",
        "ant_n": "vel_n",
        "ant_u": "vel_u",
        "time":"dts",
    },
    errors="raise"
)
inspvaa_df = inspvaa_df[
    [
        'dts', 'vel_e', 'vel_n', 'vel_u',
    ]
]

In [5]:
insstdeva_df = insstdeva_df.rename(
    columns={
        "ant_e std": "v_sde",
        "ant_n std": "v_sdn",
        "ant_u std": "v_sdu",
        "time":"dts",
    },
    errors="raise"
)
insstdeva_df = insstdeva_df[
    [
        'dts',
        'v_sde', 'v_sdn', 'v_sdu'
    ]
]

insstdeva_df["v_sden"] = 0.0
insstdeva_df["v_sdeu"] = 0.0
insstdeva_df["v_sdnu"] = 0.0


In [6]:
inspvaa_df

,dts,vel_e,vel_n,vel_u
0,7.122935e+08,0.1014,-0.6858,0.2413
1,7.122935e+08,0.0788,-0.6380,0.2734
2,7.122935e+08,0.0534,-0.5833,0.2926
3,7.122935e+08,0.0405,-0.5440,0.2950
4,7.122935e+08,0.0346,-0.5122,0.2801
...,...,...,...,...
3865082,7.123618e+08,1.0631,-0.9726,-0.1158
3865083,7.123618e+08,1.0885,-0.9804,-0.1416
3865084,7.123618e+08,1.1255,-0.9709,-0.1728
3865085,7.123618e+08,1.1578,-0.9542,-0.2087


In [7]:
insstdeva_df

,dts,v_sde,v_sdn,v_sdu,v_sden,v_sdeu,v_sdnu
0,7.122935e+08,0.0159,0.0179,0.0160,0.0,0.0,0.0
1,7.122935e+08,0.0159,0.0179,0.0160,0.0,0.0,0.0
2,7.122935e+08,0.0159,0.0179,0.0160,0.0,0.0,0.0
3,7.122935e+08,0.0159,0.0179,0.0160,0.0,0.0,0.0
4,7.122935e+08,0.0159,0.0179,0.0160,0.0,0.0,0.0
...,...,...,...,...,...,...,...
3865303,7.123618e+08,0.0147,0.0172,0.0206,0.0,0.0,0.0
3865304,7.123618e+08,0.0147,0.0172,0.0206,0.0,0.0,0.0
3865305,7.123618e+08,0.0147,0.0172,0.0206,0.0,0.0,0.0
3865306,7.123618e+08,0.0147,0.0172,0.0206,0.0,0.0,0.0


In [11]:
"""
def load_gps_solutions(
    files: List[str], time_round: int = constants.DELAY_TIME_PRECISION
) -> pd.DataFrame:
"""
gps_df = load_gps_solutions(
    files = [
        'tests/data/2022/NCL1/208/GPS_PPP/GPS_POS_FREED',
        'tests/data/2022/NCL1/209/GPS_PPP/GPS_POS_FREED',
        'tests/data/2022/NCL1/210/GPS_PPP/GPS_POS_FREED',

    ], is_l1_data=True
)
gps_df = gps_df.rename(
    columns={
        "time":"dts",
        # "xx": "sdx",
        # "yy": "sdy",
        # "zz": "sdz",
    },
    errors="raise"
)

gps_df = gps_df[
    [
        'dts', 'x', 'y', 'z', 'sdx', 'sdy', 'sdz'
    ]
]

gps_df["sdxy"] = np.sqrt(gps_df.sdx * gps_df.sdy)
gps_df["sdxz"] = np.sqrt(gps_df.sdx * gps_df.sdz)
gps_df["sdyz"] = np.sqrt(gps_df.sdy * gps_df.sdz)

gps_df

,dts,x,y,z,sdx,sdy,sdz,sdxy,sdxz,sdyz
0,712207183.0,-2.574893e+06,-3.681574e+06,4.512097e+06,2.4491,2.4489,2.4488,2.449000,2.448950,2.448850
1,712207184.0,-2.574893e+06,-3.681574e+06,4.512097e+06,2.2359,2.2357,2.2356,2.235800,2.235750,2.235650
2,712207185.0,-2.574893e+06,-3.681574e+06,4.512097e+06,2.0000,1.9998,1.9997,1.999900,1.999850,1.999750
3,712207186.0,-2.574893e+06,-3.681574e+06,4.512097e+06,1.7323,1.7320,1.7319,1.732150,1.732100,1.731950
4,712207187.0,-2.574893e+06,-3.681574e+06,4.512097e+06,1.4147,1.4144,1.4142,1.414550,1.414450,1.414300
...,...,...,...,...,...,...,...,...,...,...
193288,712400473.0,-2.575283e+06,-3.682610e+06,4.511038e+06,0.0134,0.0137,0.0158,0.013549,0.014551,0.014713
193289,712400474.0,-2.575284e+06,-3.682610e+06,4.511038e+06,0.0134,0.0137,0.0158,0.013549,0.014551,0.014713
193290,712400475.0,-2.575284e+06,-3.682610e+06,4.511038e+06,0.0134,0.0137,0.0158,0.013549,0.014551,0.014713
193291,712400476.0,-2.575284e+06,-3.682609e+06,4.511037e+06,0.0134,0.0137,0.0158,0.013549,0.014551,0.014713


In [12]:
merged_df = inspvaa_df.merge(gps_df, on="dts", how="left")
merged_df = merged_df.merge(insstdeva_df, on="dts", how="left")
merged_df = merged_df.sort_values("dts").reset_index(drop=True)

In [13]:
merged_df

,dts,vel_e,vel_n,vel_u,x,y,z,sdx,sdy,sdz,sdxy,sdxz,sdyz,v_sde,v_sdn,v_sdu,v_sden,v_sdeu,v_sdnu
0,7.122072e+08,0.1631,-1.4049,0.1295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0614,0.0650,0.0634,0.0,0.0,0.0
1,7.122072e+08,0.1743,-1.4157,0.0827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0614,0.0650,0.0634,0.0,0.0,0.0
2,7.122072e+08,0.1818,-1.4301,0.0490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0614,0.0650,0.0634,0.0,0.0,0.0
3,7.122072e+08,0.1824,-1.4465,0.0160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0614,0.0650,0.0634,0.0,0.0,0.0
4,7.122072e+08,0.1820,-1.4636,-0.0063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0614,0.0650,0.0634,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3865082,7.124005e+08,-1.3296,0.0802,0.3085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0156,0.0197,0.0197,0.0,0.0,0.0
3865083,7.124005e+08,-1.3541,0.0458,0.2389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0156,0.0197,0.0197,0.0,0.0,0.0
3865084,7.124005e+08,-1.3617,0.0044,0.1284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0156,0.0197,0.0197,0.0,0.0,0.0
3865085,7.124005e+08,-1.3508,-0.0325,0.0253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0156,0.0197,0.0197,0.0,0.0,0.0


## Selecting only first 100,000 rows to speed up comparison

In [14]:
first_pos = merged_df[~merged_df.x.isnull()].iloc[0].name
merged_df = merged_df.loc[first_pos:].reset_index(drop=True)
merged_df_small = merged_df.iloc[:1000000]

df = merged_df_small.to_records()

In [15]:
merged_df_small

,dts,vel_e,vel_n,vel_u,x,y,z,sdx,sdy,sdz,sdxy,sdxz,sdyz,v_sde,v_sdn,v_sdu,v_sden,v_sdeu,v_sdnu
0,7.122072e+08,0.1256,-1.4533,-0.4309,-2.574899e+06,-3.681586e+06,4.512084e+06,0.0414,0.0282,0.0203,0.034168,0.02899,0.023926,0.0614,0.0650,0.0634,0.0,0.0,0.0
1,7.122072e+08,0.1510,-1.4440,-0.4270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0414,0.0443,0.0552,0.0,0.0,0.0
2,7.122072e+08,0.1711,-1.4342,-0.4179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0414,0.0443,0.0552,0.0,0.0,0.0
3,7.122072e+08,0.1864,-1.4200,-0.4032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0414,0.0443,0.0552,0.0,0.0,0.0
4,7.122072e+08,0.1959,-1.4013,-0.3853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0414,0.0443,0.0552,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,7.122572e+08,-0.1597,0.5725,-0.2514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0145,0.0173,0.0197,0.0,0.0,0.0
999996,7.122572e+08,-0.1620,0.5660,-0.2675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0145,0.0173,0.0197,0.0,0.0,0.0
999997,7.122572e+08,-0.1671,0.5580,-0.2957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0145,0.0173,0.0197,0.0,0.0,0.0
999998,7.122572e+08,-0.1391,0.5430,-0.3015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0145,0.0173,0.0197,0.0,0.0,0.0


In [16]:
df

rec.array([(     0, 7.12207205e+08,  0.1256, -1.4533, -0.4309, -2574898.7626, -3681585.6405, 4512084.2874, 0.0414, 0.0282, 0.0203, 0.03416841, 0.02899, 0.02392614, 0.0614, 0.065 , 0.0634, 0., 0., 0.),
           (     1, 7.12207205e+08,  0.151 , -1.444 , -0.427 ,           nan,           nan,          nan,    nan,    nan,    nan,        nan,     nan,        nan, 0.0414, 0.0443, 0.0552, 0., 0., 0.),
           (     2, 7.12207205e+08,  0.1711, -1.4342, -0.4179,           nan,           nan,          nan,    nan,    nan,    nan,        nan,     nan,        nan, 0.0414, 0.0443, 0.0552, 0., 0., 0.),
           ...,
           (999997, 7.12257213e+08, -0.1671,  0.558 , -0.2957,           nan,           nan,          nan,    nan,    nan,    nan,        nan,     nan,        nan, 0.0145, 0.0173, 0.0197, 0., 0., 0.),
           (999998, 7.12257213e+08, -0.1391,  0.543 , -0.3015,           nan,           nan,          nan,    nan,    nan,    nan,        nan,     nan,        nan, 0.0145, 0.0173, 

## Original Kalman Algo (Should take 6-7 minutes)

In [17]:
class GNSS_filter:
    def __init__(self, row):
        dt = 5e-2

        self.Nx = 6  # number of states: pos, vel
        self.Nu = 3  # number of input var: vel

        self.X = np.zeros(
            [self.Nx, 1]
        )  # error states: pos_xyz, v_ned, eul, bias_acc, bias_gyro

        # Initial antenna location
        pos_xyz = np.array((row["x"], row["y"], row["z"])).reshape(3, 1)
        self.X[0:3] = pos_xyz
        # Initial antenna velocity
        rot, v_xyz = self.rot_vel(row)

        self.X[3:6] = v_xyz

        # Process model

        # State transition matrix
        self.F = np.identity(self.Nx)

        self.F[0:3, 3:6] = np.identity(3) * dt

        self.P = np.identity(self.Nx)  # Set initial values to 0.25?

        # Process noise matrix
        self.gnss_pos_psd = 3.125e-5
        self.vel_psd = 0.0025

        self.Q = np.zeros([self.Nx, self.Nx])

        self.updateQ(dt)

        # Position measurement noise
        self.R_position = np.identity(3)
        self.R_position[0, 0] = row.sdx**2
        self.R_position[1, 1] = row.sdy**2
        self.R_position[2, 2] = row.sdz**2

        self.R_position[0, 1] = np.sign(row.sdxy) * row.sdxy**2
        self.R_position[0, 2] = np.sign(row.sdxz) * row.sdxz**2
        self.R_position[1, 2] = np.sign(row.sdyz) * row.sdyz**2

        self.R_position[1, 0] = self.R_position[0, 1]
        self.R_position[2, 0] = self.R_position[0, 2]
        self.R_position[2, 1] = self.R_position[1, 2]

        # Velocity measurement noise
        self.R_velocity = np.identity(3) * 4e-8

        self.R_velocity[0, 1] = 1.5e-9
        self.R_velocity[0, 2] = 1.5e-9
        self.R_velocity[1, 2] = 1.5e-9

        self.R_velocity[1, 0] = self.R_velocity[0, 1]
        self.R_velocity[2, 0] = self.R_velocity[0, 2]
        self.R_velocity[2, 1] = self.R_velocity[1, 2]

        if "v_sde" in row:
            self.R_velocity[0, 0] = row.v_sde**2
            self.R_velocity[1, 1] = row.v_sdn**2
            self.R_velocity[2, 2] = row.v_sdu**2

            self.R_velocity[0, 1] = row.v_sden
            self.R_velocity[0, 2] = row.v_sdeu
            self.R_velocity[1, 2] = row.v_sdnu

            self.R_velocity[1, 0] = self.R_velocity[0, 1]
            self.R_velocity[2, 0] = self.R_velocity[0, 2]
            self.R_velocity[2, 1] = self.R_velocity[1, 2]

        self.R_velocity = rot @ self.R_velocity @ rot.transpose()

    def rot_vel(self, row):
        """
        ------------------- Rotate ENU velocity into ECEF velocity --------------------------------
                   dX = |  -sg   -sa*cg  ca*cg | | de |        de = |  -sg       cg      0 | | dX |
                   dY = |   cg   -sa*sg  ca*sg | | dn |  and   dn = |-sa*cg  -sa*sg     ca | | dY |
                   dZ = |    0    ca     sa    | | du |        du = | ca*cg   ca*sg     sa | | dZ |
        -------------------------------------------------------------------------------------------
        """
        v_enu = np.array((row["vel_e"], row["vel_n"], row["vel_u"])).reshape(3, 1)
        lat, lon, alt = pymap3d.ecef2geodetic(self.X[0], self.X[1], self.X[2])
        lat = np.deg2rad(lat)
        lam = np.deg2rad(lon)

        ca = np.cos(lat)
        sa = np.sin(lat)

        cg = np.cos(lam)
        sg = np.sin(lam)

        rot = np.zeros((3, 3))
        rot[0, 0] = -sg
        rot[0, 1] = -sa * cg
        rot[0, 2] = ca * cg
        rot[1, 0] = cg
        rot[1, 1] = -sa * sg
        rot[1, 2] = ca * sg
        rot[2, 0] = 0
        rot[2, 1] = ca
        rot[2, 2] = sa
        v_xyz = rot @ v_enu

        return rot, v_xyz

    def updateQ(self, dt):
        # Position estimation noise
        # Initial Q values from Chadwell code 3.125d-5 3.125d-5 3.125d-5 0.0025 0.0025 0.0025, assumes white noise of 2.5 cm over a second
        self.Q[0:3, 0:3] = np.identity(3) * self.gnss_pos_psd * dt

        # Velocity estimation noise (acc psd)
        self.Q[3:6, 3:6] = np.identity(3) * self.vel_psd

    def predict(self, dt):  # w is the angular rate vector
        self.F[0:3, 3:6] = np.identity(3) * dt

        self.updateQ(dt)

        self.X = self.F @ self.X
        self.P = self.F @ self.P @ self.F.transpose() + self.Q

    def updateVel_cov(self, row):
        rot, v_xyz = self.rot_vel(row)

        self.R_velocity[0, 0] = row.v_sde**2
        self.R_velocity[1, 1] = row.v_sdn**2
        self.R_velocity[2, 2] = row.v_sdu**2

        self.R_velocity[0, 1] = row.v_sden
        self.R_velocity[0, 2] = row.v_sdeu
        self.R_velocity[1, 2] = row.v_sdnu

        self.R_velocity[1, 0] = self.R_velocity[0, 1]
        self.R_velocity[2, 0] = self.R_velocity[0, 2]
        self.R_velocity[2, 1] = self.R_velocity[1, 2]
        self.R_velocity = rot @ self.R_velocity @ rot.transpose()

    def updatePosition(self, row):
        # 		pos_ned = row[['pos_n','pos_e','pos_d']].values.reshape(3,1)
        pos_xyz = np.array((row["x"], row["y"], row["z"])).reshape(3, 1)

        H = np.zeros([3, self.Nx])
        H[0:3, 0:3] = np.identity(3)

        self.R_position[0, 0] = row.sdx**2
        self.R_position[1, 1] = row.sdy**2
        self.R_position[2, 2] = row.sdz**2

        self.R_position[0, 1] = np.sign(row.sdxy) * row.sdxy**2
        self.R_position[0, 2] = np.sign(row.sdxz) * row.sdxz**2
        self.R_position[1, 2] = np.sign(row.sdyz) * row.sdyz**2

        self.R_position[1, 0] = self.R_position[0, 1]
        self.R_position[2, 0] = self.R_position[0, 2]
        self.R_position[2, 1] = self.R_position[1, 2]

        y = pos_xyz - H @ self.X
        S = H @ self.P @ H.transpose() + self.R_position
        K = (self.P @ H.transpose()) @ inv(S)
        self.X = self.X + K @ y

        I = np.identity(self.Nx)
        self.P = (I - K @ H) @ self.P

    def updateVelocity(self, row):
        # 		v_ned = row[['vel_n', 'vel_e', 'vel_d']].values.reshape(3,1)
        rot, v_xyz = self.rot_vel(row)

        H = np.zeros([3, self.Nx])
        H[0:3, 3:6] = np.identity(3)

        y = v_xyz - H @ self.X
        S = H @ self.P @ H.transpose() + self.R_velocity
        K = (self.P @ H.transpose()) @ inv(S)
        self.X = self.X + K @ y

        I = np.identity(self.Nx)
        self.P = (I - K @ H) @ self.P

    def get_states(self):
        return {
            "x": np.ndarray.item(self.X[0]),
            "y": np.ndarray.item(self.X[1]),
            "z": np.ndarray.item(self.X[2]),
            "vx": np.ndarray.item(self.X[3]),
            "vy": np.ndarray.item(self.X[4]),
            "vz": np.ndarray.item(self.X[5]),
        }

    def return_states(self):
        return self.X

    def return_covariance(self):
        return self.P

    def rts_smoother(self, Xs, Ps):
        # Rauch, Tongue, and Striebel smoother

        n, dim_x, _ = Xs.shape

        # smoother gain
        K = np.zeros((n, dim_x, dim_x))
        x, P, Pp = Xs.copy(), Ps.copy(), Ps.copy()

        i = 0
        for k in range(n - 2, -1, -1):
            Pp[k] = self.F @ P[k] @ self.F.T + self.Q  # predicted covariance

            K[k] = P[k] @ self.F.T @ inv(Pp[k])
            x[k] += K[k] @ (x[k + 1] - (self.F @ x[k]))
            P[k] += K[k] @ (P[k + 1] - Pp[k]) @ K[k].T
            i += 1
            print(str(i + 1) + "/" + str(n), end="\r")
        print("")

        return x, P, K, Pp

In [18]:
def run_filter_simulation(df):
    import time

    init = True

    results = []
    Xs = np.zeros([len(df), 6, 1])
    Ps = np.zeros([len(df), 6, 6])

    all_results = pd.DataFrame()

    # Process data through forward Kalman filter
    for i in range(len(df)):
        row = df[i]
        # print(str(i + 1) + "/" + str(len(df)), end="\r")

        if init:
            gnss_kf = GNSS_filter(row)
            init = False
            last_time = row.dts
        else:
            dt = abs(
                row.dts - last_time
            )  # This helps to stabilize the solution, abs ensures reverse filtering works.
            gnss_kf.predict(dt)
            last_time = row.dts

            # New velocity standard deviations
            if "v_sde" in row.dtype.names:
                if not np.isnan(row["v_sde"]):
                    gnss_kf.updateVel_cov(row)
            # new GNSS measurement
            if not np.isnan(row["x"]):
                gnss_kf.updatePosition(row)
            # new velocity measurement
            if not np.isnan(row["vel_e"]):
                gnss_kf.updateVelocity(row)

        X = gnss_kf.return_states()
        P = gnss_kf.return_covariance()
        Xs[i] = X
        Ps[i] = P

    print("")
    print("Smoothing results")
    x, P, K, Pp = gnss_kf.rts_smoother(Xs, Ps)

    return x, P, K, Pp

In [19]:
%%time
x_v1, P_v1, K_v1, Pp_v1 = run_filter_simulation(df)


Smoothing results
1000000/1000000
CPU times: user 12min 30s, sys: 15min 16s, total: 27min 47s
Wall time: 8min 3s


In [21]:
gps_cov_diag = P_v1.diagonal(axis1=1, axis2=2)[:, 0:3]
gps_cov_diag

array([[4.24528807e-04, 2.88290484e-04, 2.37823363e-04],
       [4.03723855e-04, 2.65831084e-04, 2.13708428e-04],
       [3.95540322e-04, 2.56615474e-04, 2.03458765e-04],
       ...,
       [8.06277259e-05, 9.58043299e-05, 1.06198513e-04],
       [8.28250351e-05, 9.81157232e-05, 1.08621428e-04],
       [8.50228616e-05, 1.00427967e-04, 1.11045521e-04]])

In [ ]:
from gnatss import constants

x_v1_df = pd.DataFrame(
    x_v1.reshape(x_v1.shape[0], -1),
    columns=[*constants.GPS_GEOCENTRIC, *constants.GPS_COV_DIAG],
)
x_v1_df[constants.GPS_TIME] = merged_df_small["dts"]
x_v1_df

## Optimized Kalman (Should take 20 seconds)

In [ ]:
from math import pi

import numpy as np
from numpy import asarray, empty_like, finfo, where
from numpy import arctan as atan
from numpy import arctan2 as atan2
from numpy import (
    cos,
    degrees,
    hypot,
    sin,
    sqrt,
    tan,
)

import numba

In [ ]:
# Process noise matrix
gnss_pos_psd = 3.125e-5
vel_psd = 0.0025

WGS84_ELL = {"name": "WGS-84 (1984)", "a": 6378137.0, "b": 6356752.31424518}
semimajor_axis = WGS84_ELL["a"]
semiminor_axis = WGS84_ELL["b"]


@numba.njit
def ecef2geodetic(
    x: float,
    y: float,
    z: float,
    semimajor_axis: float = semimajor_axis,
    semiminor_axis: float = semiminor_axis,
    eps=finfo(np.float32).eps,
) -> tuple:
    """
    convert ECEF (meters) to geodetic coordinates

    rewrite of pymap3d.ecef2geodetic to use numba for speedup...
    not as robust
    """

    x = asarray(x)
    y = asarray(y)
    z = asarray(z)

    r = sqrt(x**2 + y**2 + z**2)

    E = sqrt(semimajor_axis**2 - semiminor_axis**2)

    # eqn. 4a
    u = sqrt(0.5 * (r**2 - E**2) + 0.5 * hypot(r**2 - E**2, 2 * E * z))

    hxy = hypot(x, y)

    huE = hypot(u, E)

    # eqn. 4b
    Beta = empty_like(r)
    Beta = atan(huE / u * z / hxy)
    # eqn. 13
    Beta += ((semiminor_axis * u - semimajor_axis * huE + E**2) * sin(Beta)) / (
        semimajor_axis * huE * 1 / cos(Beta) - E**2 * cos(Beta)
    )

    # eqn. 4c
    # %% final output
    lat = atan(semimajor_axis / semiminor_axis * tan(Beta))

    # # patch latitude for float32 precision loss
    lim_pi2 = pi / 2 - eps
    lat = where(Beta >= lim_pi2, pi / 2, lat)
    lat = where(Beta <= -lim_pi2, -pi / 2, lat)

    lon = atan2(y, x)

    # eqn. 7
    cosBeta = cos(Beta)

    # patch altitude for float32 precision loss
    cosBeta = where(Beta >= lim_pi2, 0, cosBeta)
    cosBeta = where(Beta <= -lim_pi2, 0, cosBeta)

    alt = hypot(z - semiminor_axis * sin(Beta), hxy - semimajor_axis * cosBeta)

    lat = degrees(lat)
    lon = degrees(lon)

    return lat, lon, alt


@numba.njit
def predict(dt, X, P, Q, F):
    F[0:3, 3:6] = np.identity(3) * dt
    Q = updateQ(dt, Q)
    X = F @ X
    P = F @ P @ F.T + Q

    return X, P, Q, F


@numba.njit
def updateQ(dt, Q):
    # Position estimation noise
    # Initial Q values from Chadwell code 3.125d-5 3.125d-5 3.125d-5 0.0025 0.0025 0.0025, assumes white noise of 2.5 cm over a second
    Q[0:3, 0:3] = np.identity(3) * gnss_pos_psd * dt

    # Velocity estimation noise (acc psd)
    Q[3:6, 3:6] = np.identity(3) * vel_psd

    return Q


@numba.njit
def rot_vel(row, lat, lon):
    """
    ------------------- Rotate ENU velocity into ECEF velocity --------------------------------
                dX = |  -sg   -sa*cg  ca*cg | | de |        de = |  -sg       cg      0 | | dX |
                dY = |   cg   -sa*sg  ca*sg | | dn |  and   dn = |-sa*cg  -sa*sg     ca | | dY |
                dZ = |    0    ca     sa    | | du |        du = | ca*cg   ca*sg     sa | | dZ |
    -------------------------------------------------------------------------------------------
    """
    v_enu = row[1:4].copy().reshape((3, 1))
    lat = np.deg2rad(lat)
    lam = np.deg2rad(lon)

    ca = np.cos(lat)
    sa = np.sin(lat)

    cg = np.cos(lam)
    sg = np.sin(lam)

    rot = np.zeros((3, 3))
    rot[0, 0] = -sg
    rot[0, 1] = -sa * cg
    rot[0, 2] = ca * cg
    rot[1, 0] = cg
    rot[1, 1] = -sa * sg
    rot[1, 2] = ca * sg
    rot[2, 0] = 0
    rot[2, 1] = ca
    rot[2, 2] = sa
    v_xyz = rot @ v_enu

    return rot, v_xyz


@numba.njit
def update_vel_cov(row, R_velocity, rot):
    # Velocity measurement noise
    # Vel STD
    R_velocity[0, 0] = row[13] ** 2
    R_velocity[1, 1] = row[14] ** 2
    R_velocity[2, 2] = row[15] ** 2

    R_velocity[0, 1] = row[16]
    R_velocity[0, 2] = row[17]
    R_velocity[1, 2] = row[18]

    R_velocity[1, 0] = R_velocity[0, 1]
    R_velocity[2, 0] = R_velocity[0, 2]
    R_velocity[2, 1] = R_velocity[1, 2]

    R_velocity = rot @ R_velocity @ rot.T
    return R_velocity


@numba.njit
def update_rpos(row, R_position):
    R_position[0, 0] = row[7] ** 2
    R_position[1, 1] = row[8] ** 2
    R_position[2, 2] = row[9] ** 2

    R_position[0, 1] = np.sign(row[10]) * row[10] ** 2
    R_position[0, 2] = np.sign(row[11]) * row[11] ** 2
    R_position[1, 2] = np.sign(row[12]) * row[12] ** 2

    R_position[1, 0] = R_position[0, 1]
    R_position[2, 0] = R_position[0, 2]
    R_position[2, 1] = R_position[1, 2]
    return R_position


@numba.njit
def update_position(row, Nx, X, P, R_position):
    pos_xyz = row[4:7].copy().reshape((3, 1))

    H = np.zeros((3, Nx))
    H[0:3, 0:3] = np.identity(3)
    R_position = update_rpos(row, R_position)

    y = pos_xyz - H @ X
    S = H @ P @ H.T + R_position
    K = (P @ H.T) @ np.linalg.inv(S)
    X = X + K @ y

    I = np.identity(Nx)  # noqa
    P = (I - K @ H) @ P
    return X, P, R_position


@numba.njit
def update_velocity(Nx, X, P, R_velocity, v_xyz):
    H = np.zeros((3, Nx))
    H[0:3, 3:6] = np.identity(3)

    y = v_xyz - H @ X
    S = H @ P @ H.T + R_velocity
    K = (P @ H.T) @ np.linalg.inv(S)
    X = X + K @ y

    IDENTITY = np.identity(Nx)
    P = (IDENTITY - K @ H) @ P
    return X, P, R_velocity


@numba.njit
def rts_smoother(Xs, Ps, F, Q):
    # Rauch, Tongue, and Striebel smoother

    n, dim_x, _ = Xs.shape

    # smoother gain
    K = np.zeros((n, dim_x, dim_x))
    x, P, Pp = Xs.copy(), Ps.copy(), Ps.copy()

    i = 0
    for k in range(n - 2, -1, -1):
        Pp[k] = F @ P[k] @ F.T + Q  # predicted covariance

        K[k] = P[k] @ F.T @ np.linalg.inv(Pp[k])
        x[k] += K[k] @ (x[k + 1] - (F @ x[k]))
        P[k] += K[k] @ (P[k + 1] - Pp[k]) @ K[k].T
        i += 1

    return x, P, K, Pp


@numba.njit
def kalman_init(row):
    dt = 5e-2
    Nx = 6
    Nu = 3  # noqa

    # error states: pos_xyz, v_ned, eul, bias_acc, bias_gyro
    X = np.zeros((Nx, 1))
    # Initial antenna location
    pos_xyz = row[4:7].copy().reshape((3, 1))
    X[0:3] = pos_xyz

    lat, lon, _ = ecef2geodetic(X[0], X[1], X[2])
    # Initial antenna velocity
    rot, v_xyz = rot_vel(row, lat[0], lon[0])
    X[3:6] = v_xyz

    # Process model

    # State transition matrix
    F = np.identity(Nx)
    F[0:3, 3:6] = np.identity(3) * dt

    # Set initial values to 0.25?
    P = np.identity(Nx)

    # Process noise matrix
    Q = np.zeros((Nx, Nx))
    Q = updateQ(dt, Q)

    # Position measurement noise
    R_position = np.identity(3)
    R_position = update_rpos(row, R_position)

    # Velocity measurement noise
    R_velocity = np.identity(3) * 4e-8
    R_velocity[0, 1] = 1.5e-9
    R_velocity[0, 2] = 1.5e-9
    R_velocity[1, 2] = 1.5e-9
    R_velocity[1, 0] = R_velocity[0, 1]
    R_velocity[2, 0] = R_velocity[0, 2]
    R_velocity[2, 1] = R_velocity[1, 2]

    R_velocity = update_vel_cov(row, R_velocity, rot)

    return Nx, X, P, Q, F, R_position, R_velocity


@numba.njit
def run_filter_simulation(records):
    last_time = np.nan
    records_len = records.shape[0]
    Xs = np.zeros((records_len, 6, 1))
    Ps = np.zeros((records_len, 6, 6))
    n_records = len(records)
    for i in range(n_records):
        row = records[i]
        dts = row[0]
        if i == 0:
            Nx, X, P, Q, F, R_position, R_velocity = kalman_init(row)
            last_time = dts
        else:
            dt = np.abs(
                dts - last_time
            )  # This helps to stabilize the solution, abs ensures reverse filtering works.
            X, P, Q, F = predict(dt, X, P, Q, F)

            last_time = dts

            lat, lon, _ = ecef2geodetic(X[0], X[1], X[2])
            rot, v_xyz = rot_vel(row, lat[0], lon[0])

            # New velocity standard deviations
            if ~np.isnan(row[16]):
                R_velocity = update_vel_cov(row, R_velocity, rot)

            # new GNSS measurement
            if ~np.isnan(row[4]):
                X, P, R_position = update_position(row, Nx, X, P, R_position)

            # new velocity measurement
            if ~np.isnan(row[1]):
                X, P, R_velocity = update_velocity(Nx, X, P, R_velocity, v_xyz)

        Xs[i] = X
        Ps[i] = P

    x, P, K, Pp = rts_smoother(Xs, Ps, F, Q)

    return x, P, K, Pp

In [ ]:
records = merged_df_small.to_numpy()
records

In [ ]:
%%time
x_v2, P_v2, K_v2, Pp_v2 = run_filter_simulation(records)

In [ ]:
from gnatss import constants

x_v2_df = pd.DataFrame(
    x_v2.reshape(x_v2.shape[0], -1),
    columns=[*constants.GPS_GEOCENTRIC, *constants.GPS_COV_DIAG],
)
x_v2_df[constants.GPS_TIME] = merged_df_small["dts"]
x_v2_df

In [ ]:
np.array_equal(x_v1, x_v2, equal_nan=True) # Default rtol=1e-05

In [ ]:
np.allclose(x_v1, x_v2, equal_nan=True, rtol=0.0002)


In [ ]:
np.allclose(P_v1, P_v2, equal_nan=True) # Default rtol=1e-05


In [ ]:
np.allclose(K_v1, K_v2, equal_nan=True)


In [ ]:
np.allclose(Pp_v1, Pp_v2, equal_nan=True)
